### **STEP-1**. AI Solution 및 Instance 등록을 위한 준비 작업
&#x1F600; **등록 할 AI Contents 의 experimental_plan.yaml 를 alo/config/ 에 준비해 둡니다.**

&#x1F600; **가상 환경을 만들어 두고, ipykernel 을 제작해 둡니다.**     

1. ALO 의 main.py 파일이 존재하는 위치에서 아래 명령어들을 순차 실행합니다.
> conda create -n {ENV-NAME} python=3.10 \
> conda init bash \
> conda activate {ENV-NAME} \
> python main.py \
> pip install ipykernel \
> pip install requests \
> python -m ipykernel install --user --name {ENV-NAME} --display-name {IPYKERNEL-NAME}

2. 본 jupyter notebook 에서, 위에서 생성한 ipykernel 을 선택 합니다. \
   가령 tcr이라는 이름의 가상환경을 만들었다면, 아래와 같이 선택합니다.

<div style="margin: 40px">
<img src="./image/ipykernel.png" width="400">
</div>


&#x1F600; **아래 STEP들을 하나씩 실행시키면서, <u>< 사용자 입력 ></u>이라고 주석 표기된 내용을 적절히 변경해주세요.**     

----

### **STEP-2**. AI Solution 이름 선택     

#### **STEP-2-1**. AI Conductor 시스템 URI 셋팅
&#x1F600; 로그인 요청 및 시스템 담당으로부터 사용 가능한 시스템 URI를 확인합니다. 
- 고객지수플랫폼 Development 
> URI: "https://aic-kic.aidxlge.com/"
- 담당서버 테스트 환경       
> URI = "http://10.158.2.243:9999/" 
- 사외 테스트 환경 (LDAP 로그인 불가)
> URI = "https://web.aic-dev.lgebigdata.com/" 

> 시스템 URI 및 로그인 정보, ECR TAG, ICON FILE 명 등 사용자 입력부

In [8]:
#----------------------------------------#
#              사용자 입력                #
#----------------------------------------#
user_input ={
    # 로그인 정보: EP 정보로 입력해주세요
    'LOGIN_ID': 'ws.jang', # "cism-dev"
    'LOGIN_PW': 'dntjd9660@', # "cism-dev@com"
    
    # workspace 이름 
    'WORKSPACE_NAME': "magna-ws", # "cism-ws"
    
    # 시스템 URI
    'URI': "http://10.158.2.243:9999/", 
    
    # ECR에 올라갈 컨테이너 URI TAG 
    'ECR_TAG': 'latest', 

    # scripts/creating_ai_solution/image/ 밑에 UI에 표시될 아이콘 이미지 파일 (ex. icon.png) 를 업로드 해주세요. 
    # 이후 해당 아이콘 파일 명을 아래에 기록 해주세요.
    'ICON_FILE': 'icon.png'
}
#----------------------------------------#

In [91]:
import sys
del sys.modules['register_utils']

> 시스템 로그인

In [ ]:
# 시각화 용 모듈 install 
!pip install tabulate ipywidgets

In [71]:
# TODO 아무 user parameter도 선택안했을 시에도 잘 작동하는지 테스트 필요 
from register_utils import RegisterUtils
import ipywidgets as widgets

registerer = RegisterUtils(user_input)
registerer.set_yaml(pipeline = 'train')
candidate_params = registerer.set_candidate_parameters()

args_checkboxes_dict = {} 
for step_args in candidate_params:
    print('\n- step: ', step_args['step'])
    checkboxes = [widgets.Checkbox(value=False, description=arg, style={'description_width': 'initial'}) for arg in step_args['args'][0].keys()]
    args_checkboxes_dict[step_args['step']] = checkboxes
    output = widgets.VBox(children=checkboxes)
    display(output)

URI_SCOPE:  magna-ws

 << solution_metadata.yaml >> generated. - current version: v1
- step:  input


- step:  preprocess


- step:  sampling


- step:  train


In [58]:
# type(candidate_params[-1]['args'][0]['num_hpo']) == int

True

In [50]:
# import yaml 
# for params in candidate_params: 
#     yaml_data = yaml.dump(params, Dumper=yaml.SafeDumper) 
#     print(yaml_data)
#     print('----------------------------------------')

args:
- concat_dataframes: true
  drop_columns: null
  encoding: null
  groupkey_columns: null
  input_path: default_train
  time_column: null
  use_all_x: false
  x_columns:
  - input_x0
  - input_x1
  - input_x2
  - input_x3
  y_column: target
step: input

----------------------------------------
args:
- custom: {}
  mode: auto
step: preprocess

----------------------------------------
args:
- ignore_label_class: null
  label_sampling: false
  label_sampling_num: null
  label_sampling_num_type: null
  negative_target_class: null
  sampling_groupkey_columns: null
  sampling_method: random
  sampling_num: 0.8
  sampling_num_type: ratio
  sampling_type: none
step: sampling

----------------------------------------
args:
- data_split_method: cross_validate
  evaluation_metric: accuracy
  model_list:
  - lgb
  - rf
  - cb
  model_type: classification
  num_hpo: 3
  param_range:
    cb:
      max_depth:
      - 5
      - 9
      n_estimators:
      - 100
      - 500
    gbm:
      max_dept

In [72]:
# FIXME boolean은 0, 1 int로 보나?  
# args type input 
type_checkboxes_dict = {}
type_list = ['float', 'int', 'string', 'single_selection', 'multi_selection']
for step, checkboxes in args_checkboxes_dict.items(): # cbs: checkboxs
    selected_args = [c.description for c in checkboxes if c.value == True] # checked list  
    if len(selected_args) == 0: 
        continue 
    print('- step: ', step_args['step'])
    for arg in selected_args: 
        print('     - arg: ', arg)
        checkboxes = [widgets.Checkbox(value=False, description=type) for type in type_list]
        type_checkboxes_dict[arg] = checkboxes
        output = widgets.VBox(children=checkboxes)
        display(output)

- step:  train
     - arg:  concat_dataframes


     - arg:  x_columns


- step:  train
     - arg:  num_hpo


In [82]:
# FIXME checkbox 중복 선택한 경우 처리하는 로직 필요할듯 ? 

def get_arg_format(arg_type: str):
    if arg_type in ['int', 'float', 'string']:
        return ['name', 'description', 'type', 'default', 'range']
    elif arg_type in ['single_selection', 'multi_selection']: 
        return ['name', 'description', 'type', 'selectable', 'default']
    else: 
        raise ValueError(f"Unsupported type of user paramter: << {arg_type} >>") 
    
def get_text_widgets(arg_format, arg_name, arg_type):
    text_widgets = [] 
    for desc in arg_format: 
        if desc == 'name': 
            text_widgets.append(widgets.Text(description=desc, value=arg_name))
        elif desc == 'type': 
            text_widgets.append(widgets.Text(description=desc, value=arg_type))
        else: 
            text_widgets.append(widgets.Text(description=desc))
    return text_widgets

accordion_list = []
for step, checkboxes in args_checkboxes_dict.items(): 
    selected_args = [c.description for c in checkboxes if c.value == True] # checked list  
    text_widgets_list = [] 
    for arg in selected_args: 
        selected_type = [c.description for c in type_checkboxes_dict[arg] if c.value == True]
        assert len(selected_type) == 1
        arg_format = get_arg_format(selected_type[0])
        text_widgets_list.append(get_text_widgets(arg_format=arg_format, arg_name=arg, arg_type=selected_type[0]))
        
    accordion = widgets.Accordion(children=[widgets.VBox(children=text_widgets) for text_widgets in text_widgets_list])
    for idx, arg in enumerate(selected_args):
        accordion.set_title(idx, arg)
    accordion_list.append((step, accordion)) # [[step, accordion]]
    
tab_nest = widgets.Tab()
tab_nest.children = [step_accordion[1] for step_accordion in accordion_list]
for idx, step_accordion in enumerate(accordion_list): 
    tab_nest.set_title(idx, step_accordion[0])
display(tab_nest)
    

In [90]:
user_parameters = [] 
for step, accordion in accordion_list: 
    if len(accordion.children) == 0: 
        continue 
    for vbox in accordion.children: # vbox 하나가 arg 하나에 대응됨  
        args = [{tbox.name: tbox.value} for tbox in vbox] 
        user_parameters.append([{'step': step}, {'args': args}])

registerer.set_user_parameters(user_parameters)

In [44]:
# accordion_list[0][1].children[0].value

'True'

In [ ]:
# accordion = widgets.Accordion(children=[widgets.Text() for _ in range(len(selected_args[step]))])
# for idx, arg in enumerate(selected_args[step]): 
#     accordion.set_title(idx, arg)

 

In [19]:


# https://ipywidgets.readthedocs.io/en/7.x/examples/Widget%20List.html
# tab_contents = selected_data.copy() 
# children = [widgets.Text(description=name, style={'description_width': 'initial'}) for name in tab_contents]
# tab = widgets.Tab()
# tab.children = children
# for i in range(len(children)):
#     tab.set_title(i, 'arg'+str(i))
# display(tab)

['input / concat_dataframes', 'input / x_columns', 'preprocess / mode', 'sampling / sampling_groupkey_columns', 'train / shap_ratio']


In [3]:
# Generate RegisterUtils instance & login
from register_utils import RegisterUtils
registerer = RegisterUtils(user_input)

# AI Conductor Login
registerer.login()

URI_SCOPE:  magna-ws

>> Success getting cookie from AI Conductor:
 {'access-token': 'UpzeWbu9PyOdt-yhTHr2vc2j2krtrFzqL-9W7PB4Xi4'}

>> Success Login: {'result': 'OK'}


> AI Solution 이름 설정 (사용자 입력)

In [4]:
# AI Solution 이름 설정 (기존에 존재하는 중복 이름 허용X)
from IPython.display import display, HTML

# soltuion 이름 입력 받기 
display(HTML('<p style="font-size: 20px;"> < Create the name of AI Solution! > </p>'))
user_solution_name = input("- Enter the name you want to create: ")

# 기존에 존재하는 solution name과 겹치지 않는지 체크 
registerer.check_solution_name(user_solution_name)


[Success] Allowed name: << 2312121402-woosung-test >>


#### **STEE-2-2**. AI Solution Name 을 AI Conductor 에 등록합니다. 
&#x1F600; 이름이 등록되면 본 jupyter 노트북 과정이 끝날 때까지 변경이 어려 울 수 있습니다. \
변경이 필요할 경우 <b> STEP-2-1 </b> 부터 다시 실행하여 주시기 바랍니다. 
 

> 기존에 workspace에 존재하는 solution 이름을 조회하여, 사용자가 입력하는 이름이 유효한지 (고유한지) 확인합니다.

In [5]:
# 학습 용 solution_metadata.yaml 셋팅 
###################
pipeline = 'train'
###################
# check workspace (ECR, S3정보 셋팅까지 진행)
registerer.check_workspace() 
# 받아온 workspace 정보 기반으로 solution_metadata.yaml 셋팅 
registerer.set_yaml(pipeline = pipeline)

[{'id': '1b61dae3-e8f7-4b5f-9629-da0aedbb4a08', 'name': 'cism-ws', 'namespace': 'aic-ns-cism-ws', 'kubeflow_user': 'aic-user-cism-ws@aic.com', 's3_bucket_name': {'public': 's3-an2-hyunsoo-dev-aia', 'private': 's3-an2-hyunsoo-dev-cism'}, 'ecr_base_path': {'public': '086558720570.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-hyunsoo-dev/ai-solutions/public/', 'private': '086558720570.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-hyunsoo-dev/ai-solutions/cism/'}, 'execution_specs': [{'name': 'high', 'label': 'cism-ws-high', 'vcpu': 8, 'ram_gb': 32, 'gpu': 0}, {'name': 'standard', 'label': 'cism-ws-standard', 'vcpu': 2, 'ram_gb': 8, 'gpu': 0}], 'is_deleted': 0}, {'id': '93fb4d93-59a6-446b-b6ff-6af132aa4240', 'name': 'dev-ws', 'namespace': 'aic-ns-dev-ws', 'kubeflow_user': 'aic-user-dev-ws@aic.com', 's3_bucket_name': {'public': 's3-an2-hyunsoo-dev-aia', 'private': 's3-an2-hyunsoo-dev-dev'}, 'ecr_base_path': {'public': '086558720570.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-h

#### **STEE-2-3**. 솔루션 사용자가 변경 가능한 User parameters를 등록합니다.  

In [ ]:
# TODO 아무 user parameter도 선택안했을 시에도 잘 작동하는지 테스트 필요 
from register_utils import RegisterUtils
import ipywidgets as widgets

candidate_params = registerer.set_candidate_parameters()

args_checkboxes_dict = {} 
for step_args in candidate_params:
    print('\n- step: ', step_args['step'])
    checkboxes = [widgets.Checkbox(value=False, description=arg, style={'description_width': 'initial'}) for arg in step_args['args'][0].keys()]
    args_checkboxes_dict[step_args['step']] = checkboxes
    output = widgets.VBox(children=checkboxes)
    display(output)

----

#### **STEP-3**. Train 용 Sample data, train artifacts, model, icon 등록

In [6]:
# data 등록 
# s3 접근확인
registerer.s3_access_check('data')

# s3 데이터 업로드
# 이전에 이미 해당 s3 경로에 존재하던 데이터는 지워집니다.
registerer.s3_upload_data()


[INFO] AWS region: ap-northeast-2

[INFO] AWS S3 access check: OK

[INFO] Start uploading data into S3 from local folder:
 /home/ws.jang/alo_dev/tcr/alo/input/train/

Success uploading into S3: 
s3-an2-hyunsoo-dev-magna/ai-solutions/2312121402-woosung-test/v1/train/data/default_train/iris.csv

Success updating solution_metadata.yaml - << dataset_uri >> info / pipeline: train


In [7]:
# artifacts 등록 
# s3 접근확인
registerer.s3_access_check('artifacts')

# s3 model, artifacts 업로드 
# 이전에 이미 해당 s3 경로에 존재하던 object는 지워집니다.
registerer.s3_upload_artifacts()


[INFO] AWS region: ap-northeast-2

[INFO] AWS S3 access check: OK

[INFO] Start uploading << train artifacts >> into S3 from local folder:
 /home/ws.jang/alo_dev/tcr/alo/.temp_artifacts_dir/

Success uploading into S3: 
s3-an2-hyunsoo-dev-magna/ai-solutions/2312121402-woosung-test/v1/train/artifacts/train_artifacts.tar.gz

Success updating solution_metadata.yaml - << artifact_uri >> info. / pipeline: train


In [8]:
# UI에 표시될 아이콘 이미지를 s3에 업로드합니다. 
registerer.s3_upload_icon()

Success uploading into S3 path: s3-an2-hyunsoo-dev-magna/icons/2312121402-woosung-test/icon.png


----

#### **STEP-4**. Train Docker Container 제작

> ALO 작업 폴더를 현재 노트북 경로로 가져옵니다. 

In [9]:
# ALO 작업 폴더 가져오기 
registerer.set_alo()

[INFO] copy from " /home/ws.jang/alo_dev/tcr/alo/main.py "  -->  " /home/ws.jang/alo_dev/tcr/alo/scripts/creating_ai_solution/alo/ " 
[INFO] copy from " /home/ws.jang/alo_dev/tcr/alo/src "  -->  " /home/ws.jang/alo_dev/tcr/alo/scripts/creating_ai_solution/alo/src " 
[INFO] copy from " /home/ws.jang/alo_dev/tcr/alo/config "  -->  " /home/ws.jang/alo_dev/tcr/alo/scripts/creating_ai_solution/alo/config " 
[INFO] copy from " /home/ws.jang/alo_dev/tcr/alo/assets "  -->  " /home/ws.jang/alo_dev/tcr/alo/scripts/creating_ai_solution/alo/assets " 
[INFO] copy from " /home/ws.jang/alo_dev/tcr/alo/alolib "  -->  " /home/ws.jang/alo_dev/tcr/alo/scripts/creating_ai_solution/alo/alolib " 

 Success ALO directory setting.


> DOCKERFILE을 셋팅합니다. 

In [10]:
# DOCKERFILE setting
registerer.set_docker_contatiner()

Success DOCKERFILE setting. 
 - pipeline: train


> AWS ECR에 docker 등록을 위한 repository를 생성합니다. 

In [11]:
## ECR 등록
#----------------------------------------#
#              사용자 입력                #
#----------------------------------------#
docker_or_buildah = 'docker' #buildah
#----------------------------------------#

if docker_or_buildah == 'docker':
    ## docker login 실행 
    registerer.set_aws_ecr(docker=True)
    
elif docker_or_buildah == 'buildah':
    ## buildah login 실행 (docker in docker) 
    tags = [
        "Key=Company,Value=LGE",
        "Key=Owner,Value=IC360",
        "Key=HQ,Value=CDO",
        "Key=Division,Value=CDO",
        "Key=Infra Region,Value=KIC",
        "Key=Service Mode,Value=DE",
        "Key=Cost Type,Value=COMPUTING",
        "Key=Project,Value=CIS",
        "Key=Sub Project,Value=CISM",
        "Key=System,Value=AIDX"
    ]
    registerer.set_aws_ecr(docker=False, tags=tags) 

[INFO] target AWS ECR url: 
086558720570.dkr.ecr.ap-northeast-2.amazonaws.com


WARNING! Your password will be stored unencrypted in /home/ws.jang/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



[INFO] AWS ECR | docker login result: 
 Login Succeeded

[INFO] Target AWS ECR repository: 
ecr-repo-an2-hyunsoo-dev/ai-solutions/magna/2312121402-woosung-test/train/2312121402-woosung-test

[INFO] AWS ECR create-repository response: 
{
    "repository": {
        "repositoryArn": "arn:aws:ecr:ap-northeast-2:086558720570:repository/ecr-repo-an2-hyunsoo-dev/ai-solutions/magna/2312121402-woosung-test/train/2312121402-woosung-test",
        "registryId": "086558720570",
        "repositoryName": "ecr-repo-an2-hyunsoo-dev/ai-solutions/magna/2312121402-woosung-test/train/2312121402-woosung-test",
        "repositoryUri": "086558720570.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-hyunsoo-dev/ai-solutions/magna/2312121402-woosung-test/train/2312121402-woosung-test",
        "createdAt": 1702357365.0,
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": true
        }
    }
}



> Docker Build

In [12]:
# docker build 
registerer.build_docker()

Sending build context to Docker daemon  3.078MB
Step 1/12 : FROM public.ecr.aws/docker/library/python:3.10-slim-bullseye
 ---> f4856e227921
Step 2/12 : RUN apt-get update
 ---> Using cache
 ---> f526dbac1896
Step 3/12 : RUN apt-get install -y apt-utils
 ---> Using cache
 ---> 80795bb2e077
Step 4/12 : RUN apt-get install -y --no-install-recommends          build-essential          wget          ca-certificates          git          gcc     && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 30df8d89e030
Step 5/12 : ENV LC_ALL=C.UTF-8
 ---> Using cache
 ---> 9400172a5c7f
Step 6/12 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> fc96b25e90d6
Step 7/12 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Using cache
 ---> 4a310b293b84
Step 8/12 : ENV SOLUTION_PIPELINE_MODE='train'
 ---> Using cache
 ---> 6fa4a1ad9b32
Step 9/12 : ENV PATH="/framework:${PATH}"
 ---> Using cache
 ---> 3ca2fb7629ff
Step 10/12 : COPY /alo /framework
 ---> 864503ed736c
Step 11/12 : WORKDIR /framework
 ---> Running in

> ECR에 docker image를 push 합니다. 

In [13]:
registerer.docker_push()

The push refers to repository [086558720570.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-hyunsoo-dev/ai-solutions/magna/2312121402-woosung-test/train/2312121402-woosung-test]
f1a36f668753: Preparing
069eae61a1fc: Preparing
1f0af2832374: Preparing
07a72215512c: Preparing
2f33dd8c934e: Preparing
6b3d0b913afc: Preparing
277bd6c0df20: Preparing
12d601beac88: Preparing
1b6fd3ad4ce6: Preparing
6b3d0b913afc: Waiting
1b6fd3ad4ce6: Waiting
12d601beac88: Waiting
277bd6c0df20: Waiting
f1a36f668753: Pushed
1f0af2832374: Pushed
6b3d0b913afc: Pushed
2f33dd8c934e: Pushed
07a72215512c: Pushed
12d601beac88: Pushed
277bd6c0df20: Pushed
1b6fd3ad4ce6: Pushed
069eae61a1fc: Pushed
latest: digest: sha256:12ddc913e85f07d73032847954cd3e01afb2b4b536477ec1b6b5cb036456d88f size: 2215
Removing login credentials for https://index.docker.io/v1/


> solution_metadata.yaml에 container uri를 넣어줍니다.

In [14]:
registerer.set_container_uri() 

[INFO] Completes setting << container_uri >> in solution_metadata.yaml: 
086558720570.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-hyunsoo-dev/ai-solutions/magna/2312121402-woosung-test/train/2312121402-woosung-test


----

#### **STEP-5**. Training 에 사용될 User Parameters 설정

> solution_metadata.yaml에 사용자 파라미터 및 artifacts 저장 경로를 넣어줍니다. 

In [15]:
## user parameter 입력
registerer.set_candidate_parameters() # todo: selected params?
## artifact 저장 경로 지정
registerer.set_artifacts_uri()


[{self.pipeline}] Success updating << candidate_parameters >> in the solution_metadata.yaml
[INFO] Completes setting << artifact_uri >> in solution_metadata.yaml: 
s3://s3-an2-hyunsoo-dev-magna/ai-solutions/2312121402-woosung-test/v1/train/artifacts/


----

#### **STEP-6**. Training 에 사용될 Cloud resource 선택

> 컴퓨팅 자원을 선택합니다. (default: standard)

In [16]:
#----------------------------------------#
#              사용자 입력                #
#----------------------------------------#

train_resource = 'standard'

#----------------------------------------#

# 클라우드 컴퓨팅 리소스 선택 
registerer.set_resource(train_resource)


[train] Success updating << resource >> in the solution_metadata.yaml


----

#### **STEP-7**. Inference 용 Sample Data 등록

In [17]:
###################
pipeline = "inference"
###################
# inference pipeline을 solution_metadata.yaml에 추가하고, 현재 registerer의 pipeline type을 inference로 변경 
registerer.append_pipeline(pipeline)
# s3 데이터 업로드
# 이전에 있던 데이터는 지워집니다
registerer.s3_access_check('data')
registerer.s3_upload_data()


<< solution_metadata.yaml >> updated. - appended pipeline: inference

[INFO] AWS region: ap-northeast-2

[INFO] AWS S3 access check: OK

[INFO] Start uploading << data >> into S3 from local folder:
 /home/ws.jang/alo_dev/tcr/alo/input/inference/

Success uploading into S3: 
s3-an2-hyunsoo-dev-magna/ai-solutions/2312121402-woosung-test/v1/inference/data/default_inference/iris.csv

Success updating solution_metadata.yaml - << dataset_uri >> info. / pipeline: inference


In [18]:
registerer.s3_access_check('artifacts')
registerer.s3_upload_artifacts()


[INFO] AWS region: ap-northeast-2

[INFO] AWS S3 access check: OK

[INFO] Start uploading << inference artifacts >> into S3 from local folder:
 /home/ws.jang/alo_dev/tcr/alo/.temp_artifacts_dir/

Success uploading into S3: 
s3-an2-hyunsoo-dev-magna/ai-solutions/2312121402-woosung-test/v1/inference/artifacts/inference_artifacts.tar.gz

Success updating solution_metadata.yaml - << artifact_uri >> info. / pipeline: inference

[INFO] Start uploading << model >> into S3 from local folder:
 /home/ws.jang/alo_dev/tcr/alo/.temp_model_dir/

Success uploading into S3: 
s3-an2-hyunsoo-dev-magna/ai-solutions/2312121402-woosung-test/v1/train/artifacts/model.tar.gz

Success updating solution_metadata.yaml - << model_uri >> info. / pipeline: inference


----

#### **STEP-8**. Inference 용 Docker Container 제작

> DOCKERFILE을 셋팅합니다.

In [19]:
# DOCKERFILE setting
registerer.set_docker_contatiner()

Success DOCKERFILE setting. 
 - pipeline: inference


> AWS ECR에 docker 등록을 위한 repository를 생성합니다. 

In [20]:
## ECR 등록
#----------------------------------------#
#              사용자 입력                #
#----------------------------------------#
docker_or_buildah = 'docker'
#----------------------------------------#

if docker_or_buildah == 'docker':
    ## docker login 실행 
    registerer.set_aws_ecr(docker=True)
    
elif docker_or_buildah == 'buildah':
    ## buildah login 실행 (docker in docker) 
    tags = [
        "Key=Company,Value=LGE",
        "Key=Owner,Value=IC360",
        "Key=HQ,Value=CDO",
        "Key=Division,Value=CDO",
        "Key=Infra Region,Value=KIC",
        "Key=Service Mode,Value=DE",
        "Key=Cost Type,Value=COMPUTING",
        "Key=Project,Value=CIS",
        "Key=Sub Project,Value=CISM",
        "Key=System,Value=AIDX"
    ]
    registerer.set_aws_ecr(docker=False, tags=tags) 

[INFO] target AWS ECR url: 
086558720570.dkr.ecr.ap-northeast-2.amazonaws.com


WARNING! Your password will be stored unencrypted in /home/ws.jang/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



[INFO] AWS ECR | docker login result: 
 Login Succeeded

[INFO] Target AWS ECR repository: 
ecr-repo-an2-hyunsoo-dev/ai-solutions/magna/2312121402-woosung-test/inference/2312121402-woosung-test

[INFO] AWS ECR create-repository response: 
{
    "repository": {
        "repositoryArn": "arn:aws:ecr:ap-northeast-2:086558720570:repository/ecr-repo-an2-hyunsoo-dev/ai-solutions/magna/2312121402-woosung-test/inference/2312121402-woosung-test",
        "registryId": "086558720570",
        "repositoryName": "ecr-repo-an2-hyunsoo-dev/ai-solutions/magna/2312121402-woosung-test/inference/2312121402-woosung-test",
        "repositoryUri": "086558720570.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-hyunsoo-dev/ai-solutions/magna/2312121402-woosung-test/inference/2312121402-woosung-test",
        "createdAt": 1702357427.0,
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": true
        }
    }
}



> Build docker

In [21]:
# docker build 
registerer.build_docker()


Sending build context to Docker daemon  3.081MB
Step 1/12 : FROM public.ecr.aws/docker/library/python:3.10-slim-bullseye
 ---> f4856e227921
Step 2/12 : RUN apt-get update
 ---> Using cache
 ---> f526dbac1896
Step 3/12 : RUN apt-get install -y apt-utils
 ---> Using cache
 ---> 80795bb2e077
Step 4/12 : RUN apt-get install -y --no-install-recommends          build-essential          wget          ca-certificates          git          gcc     && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 30df8d89e030
Step 5/12 : ENV LC_ALL=C.UTF-8
 ---> Using cache
 ---> 9400172a5c7f
Step 6/12 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> fc96b25e90d6
Step 7/12 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Using cache
 ---> 4a310b293b84
Step 8/12 : ENV SOLUTION_PIPELINE_MODE='inference'
 ---> Using cache
 ---> 921ceb8662a1
Step 9/12 : ENV PATH="/framework:${PATH}"
 ---> Using cache
 ---> 55873e7cbb04
Step 10/12 : COPY /alo /framework
 ---> 822e9bfe6313
Step 11/12 : WORKDIR /framework
 ---> Runnin

> AWS ECR에 docker 이미지를 push 합니다. 

In [22]:
registerer.docker_push()

The push refers to repository [086558720570.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-hyunsoo-dev/ai-solutions/magna/2312121402-woosung-test/inference/2312121402-woosung-test]
955cbd1362c9: Preparing
069eae61a1fc: Preparing
1f0af2832374: Preparing
07a72215512c: Preparing
2f33dd8c934e: Preparing
6b3d0b913afc: Preparing
277bd6c0df20: Preparing
12d601beac88: Preparing
1b6fd3ad4ce6: Preparing
12d601beac88: Waiting
277bd6c0df20: Waiting
6b3d0b913afc: Waiting
1b6fd3ad4ce6: Waiting
955cbd1362c9: Pushed
1f0af2832374: Pushed
6b3d0b913afc: Pushed
2f33dd8c934e: Pushed
07a72215512c: Pushed
12d601beac88: Pushed
277bd6c0df20: Pushed
1b6fd3ad4ce6: Pushed
069eae61a1fc: Pushed
latest: digest: sha256:a85242668b56405a4856e8af30dcacaa37227086d3efec6245479aaa28cfaa59 size: 2215
Removing login credentials for https://index.docker.io/v1/


> solution_metadata.yaml에 container uri를 넣어줍니다.

In [23]:
registerer.set_container_uri() # uri도 그냥 입력되게 수정

[INFO] Completes setting << container_uri >> in solution_metadata.yaml: 
086558720570.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-hyunsoo-dev/ai-solutions/magna/2312121402-woosung-test/inference/2312121402-woosung-test


----

#### **STEP-9**. Inference 용 User Parameters 제작

> solution_metadata.yaml에 사용자 파라미터, artifacts 경로 등을 넣어줍니다.

In [24]:
registerer.set_candidate_parameters()
registerer.set_artifacts_uri()


[{self.pipeline}] Success updating << candidate_parameters >> in the solution_metadata.yaml
[INFO] Completes setting << artifact_uri >> in solution_metadata.yaml: 
s3://s3-an2-hyunsoo-dev-magna/ai-solutions/2312121402-woosung-test/v1/inference/artifacts/


> solution_metadata.yaml에 model 경로를 넣어줍니다. <u>(추론 시에만 필요합니다.)</u>

In [25]:
registerer.set_model_uri() # 주의: model은 train artifacts 경로에 존재

[INFO] Completes setting << model_uri >> in solution_metadata.yaml: 
s3://s3-an2-hyunsoo-dev-magna/ai-solutions/2312121402-woosung-test/v1/train/artifacts/


----

#### **STEP-10**. Inference 용 Resource 선택

> 컴퓨팅 자원을 선택합니다.

In [26]:
#----------------------------------------#
#              사용자 입력                #
#----------------------------------------#

inference_resource = 'standard'

#----------------------------------------#

registerer.set_resource(inference_resource)


[inference] Success updating << resource >> in the solution_metadata.yaml


----

#### **STEP-11**. Description 추가

> solution_metadata.yaml에 description을 추가합니다.

In [27]:
# [임시] 수정필요 
#----------------------------------------#
#              사용자 입력                #
#----------------------------------------#
user_description ={
    'overview': "AI Advisor Test",

    'input_data': "Test input s3 bucket",
    
    'output_data': "Test output s3 bucket",

    'user_parameters': "Test params",
    
    'algorithm': "ALO"
}
#----------------------------------------#

registerer.set_description(user_description)


<< solution_metadata.yaml >> updated. 
- solution metadata description:
 {'icon': 's3://s3-an2-hyunsoo-dev-magna/icons/2312121402-woosung-test/icon.png', 'title': '2312121402-woosung-test', 'overview': 'AI Advisor Test', 'input_data': 's3-an2-hyunsoo-dev-magnaTest input s3 bucket', 'output_data': 's3-an2-hyunsoo-dev-magnaTest input s3 bucket', 'user_parameters': 'Test params', 'algorithm': 'ALO'}


----

#### **STEP-12**. AI Solution & Instance 등록 !!

> AI Solution 등록을 진행합니다.

In [28]:
# wrangler 정보 등록 
# [임시] 변경 예정
registerer.set_wrangler()

In [29]:
#----------------------------------------#
#              사용자 입력                #
#----------------------------------------#

# Edge관련 정보 등록
edgeconductor_interface = {
            'support_labeling': True,
            
            'inference_result_datatype': 'image', # 'image'
            
            'train_datatype': 'table' # 'image'
        }

#----------------------------------------#

registerer.set_edge(edgeconductor_interface)

In [30]:
registerer.register_solution()

[INFO] AI solution register response: 
 {'id': 'db743da3-a05c-4a06-8682-425211af1735', 'name': '2312121402-woosung-test', 'scope_ws': 'magna-ws', 'version': {'id': '02d16f78-4744-4992-b61f-46022ec21bdf', 'version': 1, 'train_container_uri': '086558720570.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-hyunsoo-dev/ai-solutions/magna/2312121402-woosung-test/train/2312121402-woosung-test', 'train_container_tag': 'latest', 'inference_container_uri': '086558720570.dkr.ecr.ap-northeast-2.amazonaws.com/ecr-repo-an2-hyunsoo-dev/ai-solutions/magna/2312121402-woosung-test/inference/2312121402-woosung-test', 'inference_container_tag': 'latest', 'metadata_dict': {'version': 1, 'name': '2312121402-woosung-test', 'description': {'icon': 's3://s3-an2-hyunsoo-dev-magna/icons/2312121402-woosung-test/icon.png', 'title': '2312121402-woosung-test', 'overview': 'AI Advisor Test', 'input_data': 's3-an2-hyunsoo-dev-magnaTest input s3 bucket', 'output_data': 's3-an2-hyunsoo-dev-magnaTest input s3 bucket', '

> Solution instance 등록

In [31]:
registerer.register_solution_instance()


[INFO] AI solution interface information: 
 {'name': '2312121402-woosung-test', 'solution_version_id': '02d16f78-4744-4992-b61f-46022ec21bdf', 'workspace_name': 'magna-ws'}

[INFO] AI solution instance register response: 
 {'id': '98fd88d0-4584-4c42-b466-c631d30341dd', 'name': '2312121402-woosung-test', 'workspace_id': 'a0da0a72-3ce4-43e4-b510-b875896dcb35', 'workspace_name': 'magna-ws', 'solution_version_id': '02d16f78-4744-4992-b61f-46022ec21bdf', 'solution_version': 1, 'metadata_dict': {'version': 1, 'name': '2312121402-woosung-test', 'description': {'icon': 's3://s3-an2-hyunsoo-dev-magna/icons/2312121402-woosung-test/icon.png', 'title': '2312121402-woosung-test', 'overview': 'AI Advisor Test', 'input_data': 's3-an2-hyunsoo-dev-magnaTest input s3 bucket', 'output_data': 's3-an2-hyunsoo-dev-magnaTest input s3 bucket', 'user_parameters': 'Test params', 'algorithm': 'ALO'}, 'pipeline': [{'type': 'train', 'dataset_uri': ['s3://s3-an2-hyunsoo-dev-magna/ai-solutions/2312121402-woosung-te

----

> <u>[임시]</u> 추후 Edge Conductor UI에서 진행 예정 

> Stream 등록 

In [32]:
registerer.register_stream()

[INFO] AI solution stream register response: 
 {'id': 'fcdd6671-d658-438a-a25b-d488437d9388', 'name': '2312121402-woosung-test', 'instance_id': '98fd88d0-4584-4c42-b466-c631d30341dd', 'train_dataset_uri': 's3://s3-an2-hyunsoo-dev-magna/instances/2312121402-woosung-test/2312121402-woosung-test/train/data/', 'workspace_name': 'magna-ws', 'is_deleted': 0, 'created_at': '2023-12-12 05:04:53'}


> stream run 요청 (train, inference)

In [33]:
registerer.request_run_stream()

[INFO] Run pipeline << inference >> response: 
 {'id': '82b13d6a-7a7d-4883-a9d9-ee112d89fb26', 'name': '2312121402-woosung-test-20231212050456', 'workspace_name': 'magna-ws', 'stream_id': 'fcdd6671-d658-438a-a25b-d488437d9388', 'train_pipeline_id': 'a3bf573c-4a11-48d8-9b1b-35ac2414b581', 'status': 'running', 'train_dataset_uri': ['s3://s3-an2-hyunsoo-dev-magna/ai-solutions/2312121402-woosung-test/v1/train/data/'], 'train_artifact_uri': 's3://s3-an2-hyunsoo-dev-magna/instances/2312121402-woosung-test/fcdd6671-d658-438a-a25b-d488437d9388/train/artifacts/2023/12/12/050456.455/', 'model_version': '0', 'is_deleted': 0, 'created_at': '2023-12-12 05:04:56', 'updated_at': '2023-12-12 05:04:56'}


> pipeline status 조회 

In [35]:
registerer.get_stream_status()

Stream status: Failed


> 학습 완료된 artifacts download 

In [36]:
registerer.download_artifacts()

> download 받은 학습 artifacts로 로컬 환경에서 추론해보기

In [37]:
# 가상환경이 잘 connected 돼있는지 확인 
!which python 

/home/ws.jang/miniforge3/envs/aic_test/bin/python


In [ ]:
# 현재 작업경로 확인 
!pwd

In [ ]:
# s3로부터 다운로드받은 train artifacts를 scripts 폴더 상위 경로의 main.py랑 같은 위치로 옮기고 추론 실행 
import os
os.makedirs("./.train_artifacts",  exist_ok=True)

!tar -xvf ./train_artifacts.tar.gz -C ./.train_artifacts/
!cp -r .train_artifacts ../../
!rm -rf ./.train_artifacts

!python ../../main.py --mode inference